<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/crawling_modify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## 무조건 해야하는 부분 1
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

In [1]:
import os
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# 전체 과정 사이클

In [17]:
## colab

url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [2]:
## local

url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument('headless') # headless 모드 설정
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

# 속도 향상을 위한 옵션 해제
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(r'C:\Users\yunjc\chromedriver_win32\chromedriver.exe', options=options)

C:\Users\yunjc\AppData\Local\Temp\ipykernel_37632\3536521958.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\yunjc\chromedriver_win32\chromedriver.exe', options=options)


In [9]:
pre_df = pd.read_csv(r"G:\내 드라이브\KB인턴관련\crawl_1_euckr.csv", encoding='euc-kr')
pre_df.tail()

,category,lawName,ENG,KOR
68167,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,Article 7 (Demand for Materials Related to Con...,제7조(계약에 관한 자료 등의 요구)\n① 기획재정부장관은 정부조달협정가입국으로부터...
68168,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,Article 8 (Reporting Results of Contracts)\n(1...,제8조(계약실적보고)\n① 특례규정 제25조제1항에 따른 계약실적 보고서의 내용은 ...
68169,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDUM <Ordinance of the Ministry of Strateg...,"부칙 <제358호, 2013.8.7.>\n이 규칙은 공포한 날부터 시행한다."
68170,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDA <Ordinance of the Ministry of Strategy...,"부칙 <제360호, 2013.9.17.> (전자조달의 이용 및 촉진에 관한 법률 시..."
68171,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDUM <Ordinance of the Ministry of Strateg...,"부칙 <제492호, 2015.7.20.>\n이 규칙은 정부조달에 관한 협정 개정 의..."


In [5]:
def wait_driver(xpath, t=10):
    WebDriverWait(driver, t).until(EC.presence_of_element_located((By.XPATH, xpath)))
    
def tab_move():
    driver.switch_to.window(driver.window_handles[-1])
    ko_url = driver.current_url.replace("ENG", "KOR")
    driver.get(ko_url)
    wait_driver('//*[@id="lawViewContent"]')
    driver.switch_to.frame("lawViewContent")

def tab_close():
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("indexFrame")

def parsing_laws():
    lang_pairs = {"eng": [], "kor": []}
    for i, p in enumerate(driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')):
        if i<4: continue
        for line in p.text.split("\n"):
            if i%2==0:
                lang_pairs["eng"].append(line)
            else:
                lang_pairs["kor"].append(line)

In [ ]:
## crawler total

driver.get(url)

categories = [f'tree_{i}_a' for i in range(2, 47)]
# category loop
base_df = pd.DataFrame()
for j, c in enumerate(categories):
    category = driver.find_element(By.ID, c)
    cate_list = category.text.split(" ")
    cate_name, num_of_laws = cate_list[1], int(cate_list[2].strip('()'))
    driver.execute_script("arguments[0].click();", category)

    # pages
    driver.switch_to.frame('indexFrame')
    # pages = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.paging > a")
    pages = [f'//*[@id="container"]/div/div[3]/a[{k}]' for k in range(3, 13)]
    page_idx = 0
    complete_num = 0
    print(f"{j+1}. {cate_name} 시작...")
    while True:
    # laws loop
        wait_driver('//*[@id="1"]/td[2]')
        laws = driver.find_elements(By.CSS_SELECTOR, '#list1 > tbody > tr > td > p > a')
        page_parsing_num = 0
        for law in laws:
            law_name = law.text.split('\n')[0]
            law.send_keys(Keys.ENTER)

            tab_move()
            wait_driver('/html')

            # Parsing
            eng, kor = [], []
            contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')
            for i, c in enumerate(contents[4:-2]):
                if i%2==0:
                    eng.append(c.text)
                else:
                    kor.append(c.text)
            if len(eng)!=len(kor):
                raise Exception(f"Different Length ENG{len(eng)}, KOR{len(kor)}")
            else:
                df = pd.DataFrame({"category":[cate_name]*len(eng), "lawName":[law_name]*len(eng), "ENG":eng, "KOR":kor})
                page_parsing_num+=df.shape[0]
                # print(f"{law_name} 파싱 완료 : {len(eng)} 개")

            # fininsh parsing
            tab_close()
            base_df = pd.concat([base_df, df], axis=0)
        print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}")
        complete_num += len(laws)
        if complete_num < num_of_laws:
            driver.find_element(By.XPATH, pages[page_idx]).click()
            page_idx+=1
        else:
            break
    driver.get(url)
    wait_driver('//*[@id="tree_1"]')
driver.quit()

In [7]:
cate_pass, page_pass, law_pass = True, True, True

In [8]:
## crawler (중단 지점 부터)

driver.get(url)

categories = [f'tree_{i}_a' for i in range(2, 47)]
# category loop
base_df = pd.DataFrame()
for j, c in enumerate(categories):
    category = driver.find_element(By.ID, c)
    cate_list = category.text.split(" ")
    cate_name, num_of_laws = cate_list[1], int(cate_list[2].strip('()'))
    if cate_name=='재정ㆍ경제일반':
        cate_pass=False
        print('category 중단점 도착')
    if cate_pass: continue

    driver.execute_script("arguments[0].click();", category)

    # pages
    driver.switch_to.frame('indexFrame')
    # pages = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.paging > a")
    pages = [f'//*[@id="container"]/div/div[3]/a[{k}]' for k in range(3, 13)]
    page_idx = 0
    complete_num = 0
    print(f"{j+1}. {cate_name} 시작...")
    while True:
    # laws loop
        wait_driver('//*[@id="1"]/td[2]')
        laws = driver.find_elements(By.CSS_SELECTOR, '#list1 > tbody > tr > td > p > a')
        page_parsing_num = 0
        for law in laws:
            law_name = law.text.split('\n')[0]
            if law_name=='특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행령 특례규정':
                law_pass = False
                print('\t법령 중단점 도착\n새로운 파싱 시작')
            if law_pass: continue
            law.send_keys(Keys.ENTER)

            tab_move()
            wait_driver('/html')

            # Parsing
            eng, kor = [], []
            contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')
            for i, c in enumerate(contents[4:-2]):
                if i%2==0:
                    eng.append(c.text)
                else:
                    kor.append(c.text)
            if len(eng)!=len(kor):
                raise Exception(f"Different Length ENG{len(eng)}, KOR{len(kor)}")
            else:
                df = pd.DataFrame({"category":[cate_name]*len(eng), "lawName":[law_name]*len(eng), "ENG":eng, "KOR":kor})
                page_parsing_num+=df.shape[0]
                # print(f"{law_name} 파싱 완료 : {len(eng)} 개")

            # fininsh parsing
            tab_close()
            base_df = pd.concat([base_df, df], axis=0)
        print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}")
        complete_num += len(laws)
        if complete_num < num_of_laws:
            driver.find_element(By.XPATH, pages[page_idx]).click()
            page_idx+=1
        else:
            break
    driver.get(url)
    wait_driver('//*[@id="tree_1"]')
driver.quit()


category 중단점 도착
19. 재정ㆍ경제일반 시작...
	1페이지 파싱 완료: 0
	2페이지 파싱 완료: 0
	3페이지 파싱 완료: 0
	4페이지 파싱 완료: 0
	법령 중단점 도착
새로운 파싱 시작
	5페이지 파싱 완료: 140
	6페이지 파싱 완료: 903
20. 내국세 시작...
	1페이지 파싱 완료: 2434
	2페이지 파싱 완료: 2947
	3페이지 파싱 완료: 1033
21. 관세 시작...
	1페이지 파싱 완료: 1640
22. 담배·인삼 시작...
	1페이지 파싱 완료: 230
23. 통화ㆍ국채ㆍ금융 시작...
	1페이지 파싱 완료: 1037
	2페이지 파싱 완료: 1231
	3페이지 파싱 완료: 1701
	4페이지 파싱 완료: 1665
	5페이지 파싱 완료: 2361
	6페이지 파싱 완료: 774
24. 농업 시작...
	1페이지 파싱 완료: 1389
	2페이지 파싱 완료: 1431
	3페이지 파싱 완료: 1151
	4페이지 파싱 완료: 517
25. 축산 시작...
	1페이지 파싱 완료: 1060
	2페이지 파싱 완료: 347
26. 산림 시작...
	1페이지 파싱 완료: 1607
	2페이지 파싱 완료: 227
27. 수산 시작...
	1페이지 파싱 완료: 1203
	2페이지 파싱 완료: 487
28. 상업ㆍ무역ㆍ공업 시작...
	1페이지 파싱 완료: 1231
	2페이지 파싱 완료: 1529
	3페이지 파싱 완료: 2075
	4페이지 파싱 완료: 1286
	5페이지 파싱 완료: 1304
	6페이지 파싱 완료: 1640
	7페이지 파싱 완료: 801
29. 공업규격·계량 시작...
	1페이지 파싱 완료: 933
30. 공업소유권 시작...
	1페이지 파싱 완료: 1521
	2페이지 파싱 완료: 519
31. 에너지이용ㆍ광업 시작...
	1페이지 파싱 완료: 1449
	2페이지 파싱 완료: 491
32. 전기ㆍ가스 시작...
	1페이지 파싱 완료: 1622
	2페이지 파싱 완료: 39
33. 국토개발·도시 시작...
	1페이지 파싱 완료: 

In [13]:
total_df = pd.concat([pre_df, base_df], axis=0)
total_df.reset_index(drop=True, inplace=True)

In [ ]:
total_df.to_csv(r"G:\내 드라이브\KB인턴관련\crawl_total_local.csv", index=False, encoding='euc-kr')